# QDP: GPU-Accelerated Quantum Data Preparation

Welcome to **QDP** (Quantum Data Preparation). This library accelerates the encoding of classical data into quantum states using GPUs.

**In this tutorial, you will learn:**
1. How to install QDP on Google Colab.
2. How to initialize the QDP Engine.
3. How to encode data (Lists, NumPy, PyTorch) into quantum states.
4. **Real-world Usage**: Integrating QDP with PennyLane to train a Quantum Neural Network.

## 1. Environment Setup

First, we verify we have a GPU, install Rust (needed to compile the extension), and install the QDP package.

In [ ]:
# Check for NVIDIA GPU
!nvidia-smi

In [ ]:
# 1. Install Rust Toolchain (Required for QDP compilation)
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] += ":/root/.cargo/bin"

# 2. Install uv and clone Mahout repository
!pip install uv
!git clone https://github.com/apache/mahout.git

# 3. Install from repository root into the active Colab kernel
%cd /content/mahout
!uv pip install --system -e . --extra qdp

# 4. Notebook dependencies
!uv pip install --system torch numpy pennylane "cloudpickle>=3.0.0" "antlr4-python3-runtime==4.9.*"


## 2. Basic Usage

Now we can initialize the engine and perform our first encoding.

In [ ]:
import torch
import numpy as np
from qumat.qdp import QdpEngine

# qumat.qdp exposes a stub when native _qdp extension is unavailable.
QDP_AVAILABLE = QdpEngine.__module__ == "_qdp"
engine = None

print("Imported QdpEngine from qumat.qdp")
if not QDP_AVAILABLE:
    print("QDP native extension is not installed.")
    print("Skipping QDP execution steps. Install with: uv pip install --system -e . --extra qdp")
else:
    # Initialize engine on GPU 0
    engine = QdpEngine(0)
    print("QDP Engine initialized successfully on GPU 0")


In [ ]:
if not QDP_AVAILABLE:
    print("Skipping encoding example because QDP extension is not installed.")
else:
    # Example 1: Encode a simple Python list
    data = [0.5, 0.5, 0.5, 0.5]
    n_qubits = 2

    # Encode using amplitude encoding
    # 4 values can form a state of 2 qubits (2^2 = 4)
    qtensor = engine.encode(data, n_qubits, "amplitude")

    # Convert to PyTorch tensor (zero-copy)
    torch_tensor = torch.from_dlpack(qtensor)

    print(f"Quantum state shape: {torch_tensor.shape}")
    print(f"Quantum state data:\n{torch_tensor}")


## 3. Real-World Integration: Training a QNN with PennyLane

The true power of QDP lies in its ability to feed pre-processed quantum states directly into a Quantum Machine Learning (QML) workflow.

In this example, we will:
1.  Generate synthetic classification data.
2.  Use **QDP** to batch-encode this data into quantum states on the GPU.
3.  Feed these states into a **PennyLane** Quantum Neural Network (QNN).
4.  Train the QNN using PyTorch.

In [ ]:
import pennylane as qml
import torch.nn as nn
import torch.optim as optim

# Configuration
n_qubits = 4
batch_size = 32
n_features = 1 << n_qubits  # Amplitude encoding: 2^n features
learning_rate = 0.1
epochs = 5

# 1. Create a PennyLane Device
# Use 'default.qubit' (CPU) or 'lightning.gpu' (if installed) for simulation.
# QDP handles the heavy lifting of state preparation on GPU first.
dev = qml.device("default.qubit", wires=n_qubits)

# 2. Define the QNode (Quantum Circuit)
# This takes a pre-calculated state vector as input
@qml.qnode(dev, interface="torch")
def qnn_circuit(inputs, weights):
    # Initialize the qubit register with the state from QDP
    qml.StatePrep(inputs, wires=range(n_qubits))
    
    # Trainable Variational Layers
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    
    # Measure expectation value
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

print("PennyLane QNode defined successfully.")

In [ ]:
if not QDP_AVAILABLE:
    print("Skipping QNN training example because QDP extension is not installed.")
else:
    # 3. Data preparation (synthetic)
    # Generate random features and binary labels
    input_data = np.random.rand(batch_size, n_features).astype(np.float64)

    # Important: use float64 regarding the dtype mismatch error (Float vs Double)
    labels = torch.randint(0, 2, (batch_size,)).to(torch.float64)

    # 4. QDP encoding (the acceleration step)
    print("Encoding data on GPU with QDP...")
    qtensor_batch = engine.encode(input_data, n_qubits, "amplitude")
    # Converting to PyTorch tensor (on GPU)
    train_states_gpu = torch.from_dlpack(qtensor_batch)


    # 5. Define PyTorch model using the QNode
    weight_shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=n_qubits)
    # Initialize weights as float64 to match input precision
    weights = torch.nn.Parameter(torch.rand(weight_shape, dtype=torch.float64))
    optimizer = optim.Adam([weights], lr=learning_rate)
    loss_fn = nn.MSELoss()  # Simple MSE for demonstration

    print(f"Starting training for {epochs} epochs...")

    # 6. Training loop
    for epoch in range(epochs):
        optimizer.zero_grad()

        # Forward pass: feed QDP states into PennyLane circuit
        # We sum the Z-expectation values to get a single prediction per sample
        predictions = torch.stack([torch.sum(torch.stack(qnn_circuit(state, weights))) for state in train_states_gpu])

        # Normalize predictions to [0, 1] for dummy classification (sigmoid-like)
        predictions = torch.sigmoid(predictions)

        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs} | Loss: {loss.item():.4f}")

    print("Training complete!")
